In [1]:
! pip install transformers

     |████████████████████████████████| 2.3MB 2.9MB/s 
     |████████████████████████████████| 3.3MB 19.5MB/s 
     |████████████████████████████████| 901kB 35.5MB/s 


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import glob
from bert_train_fox import train_bert



## Training on fox news dataset

In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os

#### Initialize bert classification model

In [4]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [5]:
import os

In [6]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                        trainable=True, 
                                                        num_labels=2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)



Read hate dataset and convert it into train and test

In [8]:
df = pd.read_csv("data/fox_news.csv")
df = df.drop(columns=['Unnamed: 0'])
# df['comment'] = df['comment'].str.strip()

In [9]:
df['label'] = df['label'].replace({2:1})
df.head()

,comment,label
0,Merkel would never say NO,1
1,"Expect more and more women to be asking .. ""wh...",1
2,Groping people in public wasn't already illega...,0
3,"Merkel, possible the only person in charge who...",1
4,"They know very well, no means NO They need to ...",1


In [10]:
def get_dataset(df, seed, test_size):
    return train_test_split(df, test_size=test_size, random_state=seed, shuffle=True)

In [11]:
train, test = get_dataset(df, 11, 0.2)

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1210 entries, 941 to 1104
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  1210 non-null   object
 1   label    1210 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 28.4+ KB


In [13]:
train = train.append(test)
train.count()

comment    1513
label      1513
dtype: int64

In [14]:
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']


In [15]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [16]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.batch(32)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])



In [18]:
hist = model.fit(train_data, epochs=6, validation_data=validation_data, callbacks=[cp_callback])

Epoch 1/6
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [19]:
preds = model.predict(validation_data)

In [20]:
print(classification_report(test['LABEL_COLUMN'],np.argmax(preds[0],axis=1)))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       229
           1       1.00      0.96      0.98        74

    accuracy                           0.99       303
   macro avg       0.99      0.98      0.99       303
weighted avg       0.99      0.99      0.99       303





---



---



---



---



## Cross validation by loading above training set

In [21]:
# perform cross folding
def load_movies_to_df(path):
    df_movies = []

    for filename in glob.glob(path + '*.csv'):
        df_movies.append(pd.read_csv(filename))

    return df_movies

In [22]:
df_movies = load_movies_to_df('movies/for_training/')
classification_report = []
df_main = pd.DataFrame()

In [23]:
for i in range(len(df_movies)):
    df_train = pd.concat(df_movies[0:i] + df_movies[i + 1:])
    df_test = df_movies[i]

    df_train['majority_answer'] = df_train['majority_answer'].replace({2:1})
    df_test['majority_answer'] = df_test['majority_answer'].replace({2:1})

    train_movies = df_train['movie_name'].unique()
    test_movie = df_test['movie_name'].unique()
    print(','.join(train_movies))
    print(test_movie[0])
    classification_report.append(train_bert(df_train, df_test))
    # classification_report.append(x)
    print('Train movies: ', str(','.join(train_movies)))
    print('Test movie: ', str(test_movie[0]))
    print('Classification report: \n', classification_report[i])
    print('------------------------------------------------')

    df_cr = pd.DataFrame(classification_report[i]).transpose()
    df_cr['movie_train'] =  str(','.join(train_movies))
    df_cr['movie_test'] = str(test_movie[0])
    df_cr.to_csv('classification_reports/'+str(test_movie[0])+'.csv')
    df_main = df_main.append(df_cr)

South_Park,Django_Unchained,Pulp_Fiction,AmerricanHistoryX,TheWolfofWallStreet
BlacKkKlansman


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/6
283/283 [==============================] - 439s 1s/step - loss: 0.2421 - accuracy: 0.9108
Epoch 2/6
283/283 [==============================] - 415s 1s/step - loss: 0.0984 - accuracy: 0.9651
Epoch 3/6
283/283 [==============================] - 418s 1s/step - loss: 0.0534 - accuracy: 0.9837
Epoch 4/6
283/283 [==============================] - 419s 1s/step - loss: 0.0379 - accuracy: 0.9888
Epoch 5/6
283/283 [==============================] - 420s 1s/step - loss: 0.0304 - accuracy: 0.9916
Epoch 6/6
283/283 [==============================] - 419s 1s/step - loss: 0.0250 - accuracy: 0.9932
predicting
Train movies:  South_Park,Django_Unchained,Pulp_Fiction,AmerricanHistoryX,TheWolfofWallStreet
Test movie:  BlacKkKlansman
Classification report: 
 {'0': {'precision': 0.9588116137744767, 'recall': 0.9712722298221614, 'f1-score': 0.9650016989466531, 'support': 1462}, '1': {'precision': 0.7439024390243902, 'recall': 0.6666666666666666, 'f1-score': 0.7031700288184437, 'support': 183}, 'acc

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/6
302/302 [==============================] - 471s 1s/step - loss: 0.1927 - accuracy: 0.9350
Epoch 2/6
302/302 [==============================] - 444s 1s/step - loss: 0.0930 - accuracy: 0.9669
Epoch 3/6
302/302 [==============================] - 444s 1s/step - loss: 0.0484 - accuracy: 0.9866
Epoch 4/6
302/302 [==============================] - 444s 1s/step - loss: 0.0334 - accuracy: 0.9900
Epoch 5/6
302/302 [==============================] - 445s 1s/step - loss: 0.0254 - accuracy: 0.9927
Epoch 6/6
302/302 [==============================] - 444s 1s/step - loss: 0.0185 - accuracy: 0.9943
predicting
Train movies:  BlacKkKlansman,Django_Unchained,Pulp_Fiction,AmerricanHistoryX,TheWolfofWallStreet
Test movie:  South_Park
Classification report: 
 {'0': {'precision': 0.9525909592061742, 'recall': 0.9707865168539326, 'f1-score': 0.9616026711185309, 'support': 890}, '1': {'precision': 0.8129496402877698, 'recall': 0.7243589743589743, 'f1-score': 0.7661016949152544, 'support': 156}, 'accu

...

Epoch 1/6
284/284 [==============================] - 442s 1s/step - loss: 0.1947 - accuracy: 0.9300
Epoch 2/6
284/284 [==============================] - 416s 1s/step - loss: 0.0899 - accuracy: 0.9652
Epoch 3/6
284/284 [==============================] - 415s 1s/step - loss: 0.0442 - accuracy: 0.9839
Epoch 4/6
284/284 [==============================] - 415s 1s/step - loss: 0.0332 - accuracy: 0.9902
Epoch 5/6
284/284 [==============================] - 416s 1s/step - loss: 0.0238 - accuracy: 0.9924
Epoch 6/6
284/284 [==============================] - 416s 1s/step - loss: 0.0200 - accuracy: 0.9930
predicting
Train movies:  BlacKkKlansman,South_Park,Django_Unchained,AmerricanHistoryX,TheWolfofWallStreet
Test movie:  Pulp_Fiction
Classification report: 
 {'0': {'precision': 0.9813664596273292, 'recall': 0.9482370592648162, 'f1-score': 0.964517359786341, 'support': 1333}, '1': {'precision': 0.7934131736526946, 'recall': 0.916955017301038, 'f1-score': 0.8507223113964686, 'support': 289}, 'accur

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/6
286/286 [==============================] - 444s 1s/step - loss: 0.2059 - accuracy: 0.9260
Epoch 2/6
286/286 [==============================] - 419s 1s/step - loss: 0.0959 - accuracy: 0.9654
Epoch 3/6
286/286 [==============================] - 418s 1s/step - loss: 0.0525 - accuracy: 0.9811
Epoch 4/6
286/286 [==============================] - 418s 1s/step - loss: 0.0329 - accuracy: 0.9886
Epoch 5/6
286/286 [==============================] - 418s 1s/step - loss: 0.0266 - accuracy: 0.9929
Epoch 6/6
286/286 [==============================] - 419s 1s/step - loss: 0.0209 - accuracy: 0.9940
predicting
Train movies:  BlacKkKlansman,South_Park,Django_Unchained,Pulp_Fiction,TheWolfofWallStreet
Test movie:  AmerricanHistoryX
Classification report: 
 {'0': {'precision': 0.9644208932626798, 'recall': 0.9769938650306749, 'f1-score': 0.9706666666666667, 'support': 1304}, '1': {'precision': 0.8770491803278688, 'recall': 0.8199233716475096, 'f1-score': 0.8475247524752475, 'support': 261}, 'acc

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/6
239/239 [==============================] - 377s 1s/step - loss: 0.2098 - accuracy: 0.9206
Epoch 2/6
239/239 [==============================] - 352s 1s/step - loss: 0.0933 - accuracy: 0.9663
Epoch 3/6
239/239 [==============================] - 352s 1s/step - loss: 0.0569 - accuracy: 0.9781
Epoch 4/6
239/239 [==============================] - 352s 1s/step - loss: 0.0316 - accuracy: 0.9909
Epoch 5/6
239/239 [==============================] - 352s 1s/step - loss: 0.0301 - accuracy: 0.9909
Epoch 6/6
239/239 [==============================] - 351s 1s/step - loss: 0.0237 - accuracy: 0.9925
predicting
Train movies:  BlacKkKlansman,South_Park,Django_Unchained,Pulp_Fiction,AmerricanHistoryX
Test movie:  TheWolfofWallStreet
Classification report: 
 {'0': {'precision': 0.9833333333333333, 'recall': 0.9777687954729184, 'f1-score': 0.9805431698419131, 'support': 2474}, '1': {'precision': 0.9087893864013267, 'recall': 0.9303904923599321, 'f1-score': 0.9194630872483222, 'support': 589}, 'acc

In [24]:
df_main.to_csv('cross_fold_results_foxnews_finetune_6epochs.csv')

In [25]:
print(df_main)

              precision  ...           movie_test
0              0.958812  ...       BlacKkKlansman
1              0.743902  ...       BlacKkKlansman
accuracy       0.937386  ...       BlacKkKlansman
macro avg      0.851357  ...       BlacKkKlansman
weighted avg   0.934904  ...       BlacKkKlansman
0              0.952591  ...           South_Park
1              0.812950  ...           South_Park
accuracy       0.934034  ...           South_Park
macro avg      0.882770  ...           South_Park
weighted avg   0.931765  ...           South_Park
0              0.982973  ...     Django_Unchained
1              0.772727  ...     Django_Unchained
accuracy       0.956497  ...     Django_Unchained
macro avg      0.877850  ...     Django_Unchained
weighted avg   0.959385  ...     Django_Unchained
0              0.981366  ...         Pulp_Fiction
1              0.793413  ...         Pulp_Fiction
accuracy       0.942663  ...         Pulp_Fiction
macro avg      0.887390  ...         Pulp_Fiction


In [26]:
len(classification_report[0])

5

In [27]:
df_main.head()

,precision,recall,f1-score,support,movie_train,movie_test
0,0.958812,0.971272,0.965002,1462.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
1,0.743902,0.666667,0.703170,183.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
accuracy,0.937386,0.937386,0.937386,0.937386,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
macro avg,0.851357,0.818969,0.834086,1645.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
weighted avg,0.934904,0.937386,0.935874,1645.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman


In [28]:
def get_precision_recall_f1(category, result_df):
    precision = result_df[result_df.label==category].precision.mean()
    recall = result_df[result_df.label==category].recall.mean()
    f1 = result_df[result_df.label==category]['f1-score'].mean()
    
    return {'label': category, 'precision': precision, 'recall': recall, 'f1': f1}

In [29]:
df_cv= pd.read_csv('cross_fold_results_foxnews_finetune_6epochs.csv')

In [30]:
df_cv = df_cv.rename(columns={'Unnamed: 0': 'label', 'b': 'Y'})
df_cv.head()

,label,precision,recall,f1-score,support,movie_train,movie_test
0,0,0.958812,0.971272,0.965002,1462.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
1,1,0.743902,0.666667,0.703170,183.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
2,accuracy,0.937386,0.937386,0.937386,0.937386,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
3,macro avg,0.851357,0.818969,0.834086,1645.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman
4,weighted avg,0.934904,0.937386,0.935874,1645.000000,"South_Park,Django_Unchained,Pulp_Fiction,Amerr...",BlacKkKlansman


In [31]:
normal_dict = get_precision_recall_f1('0', df_cv)
offensive_dict = get_precision_recall_f1('1',df_cv)

In [32]:
df_result = pd.DataFrame([normal_dict, offensive_dict])
df_result

,label,precision,recall,f1
0,0,0.970583,0.968804,0.969607
1,1,0.818139,0.820940,0.817382


In [33]:
for cr in classification_report:
  print(cr)

{'0': {'precision': 0.9588116137744767, 'recall': 0.9712722298221614, 'f1-score': 0.9650016989466531, 'support': 1462}, '1': {'precision': 0.7439024390243902, 'recall': 0.6666666666666666, 'f1-score': 0.7031700288184437, 'support': 183}, 'accuracy': 0.9373860182370821, 'macro avg': {'precision': 0.8513570263994334, 'recall': 0.8189694482444141, 'f1-score': 0.8340858638825484, 'support': 1645}, 'weighted avg': {'precision': 0.9349037846077497, 'recall': 0.9373860182370821, 'f1-score': 0.935873920446068, 'support': 1645}}
{'0': {'precision': 0.9525909592061742, 'recall': 0.9707865168539326, 'f1-score': 0.9616026711185309, 'support': 890}, '1': {'precision': 0.8129496402877698, 'recall': 0.7243589743589743, 'f1-score': 0.7661016949152544, 'support': 156}, 'accuracy': 0.9340344168260039, 'macro avg': {'precision': 0.882770299746972, 'recall': 0.8475727456064535, 'f1-score': 0.8638521830168926, 'support': 1046}, 'weighted avg': {'precision': 0.9317649116428175, 'recall': 0.9340344168260039,